# Sentiment Classification Lab

In [1]:
from appvocai-discover.analysis.sentiment import SentimentClassifierSemiSupervised, SentimentClassifierConfig
from appvocai-discover.config.training import TrainingConfig, FineTuningConfig
from appvocai-discover.shared.io import IOService

/home/john/anaconda3/envs/appvocai-discover/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/john/anaconda3/envs/appvocai-discover/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
fp = "data/01_exp/review.csv"
model_name = "finiteautomata/bertweet-base-sentiment-analysis"

## Data and Configurations

In [3]:
reviews = IOService.read(fp)
classifier_config = SentimentClassifierConfig(target_dataset_size=2**5, sample_size=2**4)
training_config = TrainingConfig()
finetune_config = FineTuningConfig()

## Full Run

In [4]:
clf = SentimentClassifierSemiSupervised(reviews=reviews, model_name=model_name, 
                                        classifier_config=classifier_config, 
                                        training_config=training_config, 
                                        finetune_config=finetune_config)
stats = clf.run()

/home/john/anaconda3/envs/appvocai-discover/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-06 20:04:25,845:sample_reviews:INFO:Sample reviews method returning dataset of 16 observations.
The model 'PeftModelForSequenceClassification' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'D

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.006976,1.000000
2,No log,0.002411,1.000000
3,No log,0.001534,1.000000


2024-05-06 20:05:46,118:train:INFO:Training complete


2024-05-06 20:05:47,553:sample_reviews:INFO:Sample reviews method returning dataset of 16 observations.


{'train': {'train_loss': 0.0015338632510975003, 'train_accuracy': 1.0, 'train_runtime': 0.9657, 'train_samples_per_second': 9.32, 'train_steps_per_second': 1.036, 'epoch': 3.0}, 'validation': {'eval_loss': 0.0009233011514879763, 'eval_accuracy': 1.0, 'eval_runtime': 0.3545, 'eval_samples_per_second': 8.463, 'eval_steps_per_second': 2.821, 'epoch': 3.0}}


...

The model 'PeftModelForSequenceClassification' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification',

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000103,1.000000
2,No log,0.000045,1.000000
3,No log,0.000036,1.000000


2024-05-06 20:11:42,046:train:INFO:Training complete


2024-05-06 20:12:10,872:save_model:INFO:Saved model to models//sentiment-analysis/sentiment-analysis-lora-32-samples-0.85-confidence/.


{'train': {'train_loss': 3.5803153878077865e-05, 'train_accuracy': 1.0, 'train_runtime': 28.0385, 'train_samples_per_second': 1.213, 'train_steps_per_second': 0.071, 'epoch': 3.0}, 'validation': {'eval_loss': 2.8450662284740247e-05, 'eval_accuracy': 1.0, 'eval_runtime': 0.7718, 'eval_samples_per_second': 11.66, 'eval_steps_per_second': 1.296, 'epoch': 3.0}}


In [5]:
print(stats)
clf.evaluate()



                                      SentimentClassifierStats                                      
                               Target Dataset Size | 32
                                        Start Time | 2024-05-06 20:04:25.799175
                                          End Time | 2024-05-06 20:12:10.862763
                                        Iterations | 3
                              Current Dataset Size | 82
                              Iteration Start Time | 2024-05-06 20:08:44.357733
                                Iteration End Time | 2024-05-06 20:12:10.862734
           Average Samples Augmented Per Iteration | 27.3
                         Average Iteration Runtime | 155.02116133333334
                                     Total Runtime | 465 days, 1:31:34.003200
                               Total Training Time | 465.063484
                                            Metric | validation_accuracy
                                     Average Score | 1.0
        

TypeError: SentimentClassifierSemiSupervised.evaluate() missing 1 required positional argument: 'tokenized_dataset'